# Exploration of Merging Data Files

This notebook was built to explore how to best merge the Cisco Umbrella 1m Domains file and the Bambenek Consulting DGA High-Confidence Feed.

In [15]:
# Libraries
import zipfile
import numpy as np
import pandas as pd
import re
import os
import time
import sys
import lstm_binary
import imp

In [10]:
merged_csv = '/home/nscsekhar/team_cyber/data/local/staging/merged-2018-11-23.csv'
df = pd.read_csv(merged_csv)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X = df['domain']
Y = df['dga']

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=23)

In [22]:
imp.reload(lstm_binary)
train_model = lstm_binary.LSTMBinary()
train_model.train(X_train, Y_train)

Epoch 1/10
1463529/1463529 [==============================] - 28s 19us/step - loss: 0.2928
Epoch 2/10
1463529/1463529 [==============================] - 27s 18us/step - loss: 0.1490
Epoch 3/10
1463529/1463529 [==============================] - 27s 18us/step - loss: 0.1110
Epoch 4/10
1463529/1463529 [==============================] - 27s 18us/step - loss: 0.1019
Epoch 5/10
1463529/1463529 [==============================] - 27s 18us/step - loss: 0.0950
Epoch 6/10
1463529/1463529 [==============================] - 27s 18us/step - loss: 0.0886
Epoch 7/10
1463529/1463529 [==============================] - 27s 18us/step - loss: 0.0826
Epoch 8/10
1463529/1463529 [==============================] - 27s 18us/step - loss: 0.1666
Epoch 9/10
1463529/1463529 [==============================] - 27s 18us/step - loss: 0.0779
Epoch 10/10
1463529/1463529 [==============================] - 27s 18us/step - loss: 0.0736


In [23]:
PROJECT_ROOT = "./"
BINARY_TOKENIZER_FILE = PROJECT_ROOT + "saved_models/binary_tokenizer.pkl"
BINARY_CATEGORIES_FILE = PROJECT_ROOT + "saved_models/binary_categories.pkl"
BINARY_MODEL_JSON = PROJECT_ROOT + "saved_models/binary_LSTM.json"
BINARY_MODEL_H5 = PROJECT_ROOT + "saved_models/binary_LSTM.h5"

train_model.save(BINARY_TOKENIZER_FILE, BINARY_MODEL_JSON, BINARY_MODEL_H5)

In [24]:
testmodel = lstm_binary.LSTMBinary()
testmodel.load(BINARY_TOKENIZER_FILE, BINARY_MODEL_JSON, BINARY_MODEL_H5)

SAVED BINARY MODEL IS NOW LOADED!


In [25]:
urllist = ["www.google.com", "www.netflix.com", "plvklpgwivery.com"]
urltypes = testmodel.predict(urllist)
print("URL type:", urltypes)

['www.google.com', 'www.netflix.com', 'plvklpgwivery.com']
URL type: [[1]
 [1]
 [0]]


In [ ]:
urltypes = testmodel.predict(X_test)

1422594                                  luhrmeumeywoepk.biz
256180                               tysonscore.cloudapp.net
426746                                       m22.dlqmaup.com
208743                                  studio.inmobicdn.net
26410                                    media.allauthor.com
1805055                                 1728695480.localhost
1386216                                    emnauxhidifgwd.su
846701                            free.theastrologyguide.com
1565510                                    ibsomdxungauro.cc
1282783                                     onvojihgpjvt.com
21555      cluster95-elbwpeel-rqn4axbbh0ly-1644986947.us-...
220613                                      cdn.mngepvra.com
799158                             www.itcratingservices.com
1819789                                 2067050032.localhost
1068610                                           mintos.com
590967                  doc-08-a8-docs.googleusercontent.com
857787                  